In [1]:
# Main import block
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO

# Sneaky scrapper
import cfscrape

# Upload to google sheets
import gspread
import numpy as np
import pandas as pd
import requests

# Upload to yandex
import yadisk
from bs4 import BeautifulSoup

# import df2gspread as d2g
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

warnings.filterwarnings("ignore")

In [22]:
################################################################
####################   CB_metalls ##############################
################################################################


async def cb_metalls():
    try:
        # Первичные гиперпараметры
        current_year = datetime.now().year

        url = f"https://www.cbr.ru/hd_base/metall/metall_base_new/?UniDbQuery.Posted\
        =True&UniDbQuery.From=01.01.2022&UniDbQuery.To=31.12.{current_year}&UniDbQuery.Gold\
        =true&UniDbQuery.Silver=true&UniDbQuery.Platinum=true&UniDbQuery.Palladium\
        =true&UniDbQuery.so=1"

        # Запуск скраппера
        scraper = cfscrape.create_scraper()
        scraped_data = scraper.get(url)

        # Первичное получение данных
        metalls = pd.read_html(scraped_data.text.replace(",", "."))[0]

        # Переименовывание колонок
        metalls = metalls.rename(
            columns={
                metalls.columns[0]: "date",
                metalls.columns[1]: "gold",
                metalls.columns[2]: "silver",
                metalls.columns[3]: "platinum",
                metalls.columns[4]: "palladium",
            }
        )

        # Приведение данных к необходимому формату
        metalls["date"] = pd.to_datetime(metalls["date"], format="%d.%m.%Y")
        metalls["gold"] = pd.to_numeric(metalls["gold"].str.replace(" ", ""))
        metalls["silver"] = pd.to_numeric(metalls["silver"])
        metalls["platinum"] = pd.to_numeric(metalls["platinum"].str.replace(" ", ""))
        metalls["palladium"] = pd.to_numeric(metalls["palladium"].str.replace(" ", ""))

        metalls = metalls.sort_values("date").reset_index(drop=True)

        with pd.ExcelWriter(
            "./data/cb_metalls.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            metalls.to_excel(writer, sheet_name="cb_metalls")

        print("CB_metalls parsing is DONE!")
        display(metalls.tail(10))

    except Exception as error:
        print(error)


In [23]:
await cb_metalls()

CB_metalls parsing is DONE!


,date,gold,silver,platinum,palladium
0,2024-03-30,6575.81,72.87,2693.46,3020.12
1,2024-04-02,6570.50,72.82,2691.28,3017.68
2,2024-04-03,6587.16,73.00,2698.11,3025.33
3,2024-04-04,6726.42,76.19,2747.60,3017.91
4,2024-04-05,6766.80,77.89,2756.99,3000.34
5,2024-04-06,6814.50,80.22,2778.10,3042.53
6,2024-04-09,6841.74,79.71,2747.35,2970.59
